In [2]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [3]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("classification_data/attack_img/attack_data.csv")
    shuffled_df = df.sample(len(df))

    print(shuffled_df.head)
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"classification_data/attack_img/{entry[0]}")

        prepared_y = np.delete(entry,0) * 3
        print(f'before: {prepared_y}')
        if prepared_y[0] <= 960:
            prepared_y[0] =  -(960 - prepared_y[0])
        else:
            prepared_y[0] = prepared_y[0]-960
            
        if prepared_y[1] <= 540:
            prepared_y[1] = - (540 - prepared_y[1])
        else:
            prepared_y[1] = prepared_y[1] - 540
        print(prepared_y)
        X.append(entry_img)
        y.append(prepared_y)

    
    X = np.asarray(X)
    y = np.asarray(y).astype(np.float32)
    return X, y


In [10]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

    return model

model = compile_model()

In [11]:
model.compile(optimizer='adam',
              loss='mean_squared_error')


In [4]:
X, y = create_input()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)


<bound method NDFrame.head of            filename    x    y
180  attack_180.jpg  355   48
236  attack_236.jpg  262   72
301  attack_301.jpg  409  136
496  attack_496.jpg  346   46
652  attack_652.jpg  356  218
..              ...  ...  ...
390  attack_390.jpg  437   18
619  attack_619.jpg  212  254
138  attack_138.jpg  362  105
435  attack_435.jpg  340  122
85    attack_85.jpg  324   93

[838 rows x 3 columns]>
before: [1065 144]
[105 -396]
before: [786 216]
[-174 -324]
before: [1227 408]
[267 -132]
before: [1038 138]
[78 -402]
before: [1068 654]
[108 114]
before: [948 576]
[-12 36]
before: [1113 480]
[153 -60]
before: [339 420]
[-621 -120]
before: [1152 312]
[192 -228]
before: [573 300]
[-387 -240]
before: [426 69]
[-534 -471]
before: [1350 375]
[390 -165]
before: [1281 195]
[321 -345]
before: [1314 336]
[354 -204]
before: [1146 312]
[186 -228]
before: [1110 333]
[150 -207]
before: [1293 189]
[333 -351]
before: [1158 252]
[198 -288]
before: [1032 744]
[72 204]
before: [1128 564]
[168 

In [12]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))


Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 922ms/step - loss: 17856638.0000 - val_loss: 102896.2891
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 913ms/step - loss: 128382.6484 - val_loss: 74028.3047
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 19s 891ms/step - loss: 63658.9844 - val_loss: 50796.3633
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 47908.5781 - val_loss: 52772.2617
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 41716.8555 - val_loss: 48890.6641
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 37712.1016 - val_loss: 52289.8516
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 32149.2520 - val_loss: 49332.4570
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 26663.2734 - val_loss: 47360.6289
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 19549.6016 - val_loss: 50309.3164
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 18146.4785 - val_loss: 50945.3945
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 19236.

In [13]:
model.save('mouse_attack_fhd_normalized.h5')

In [6]:
model_rap = tf.keras.models.load_model('mouse_attack_fhd_normalized2.h5')


for n in range(20):
    x_real, y_real = int(y_val[n][0]), int(y_val[n][1])
    x_r1, y_r1 = x_real, y_real
    img = np.asarray(X_val[n])
    
    if x_real <= 960:
        x_real =  960 + x_real
    else:
        x_real = x_real + 960
        
    if y_real <= 540:
        y_real = 540 + y_real 
    else:
        y_real = y_real + 540
    
    tbp = np.expand_dims(img, axis=0)
    img = cv2.resize(img,(1920,1080))
    predictions = model_rap.predict(tbp)
    
    x, y = int(predictions[0][0]), int(predictions[0][1])
    
    if x <= 960:
        x =  960 + x
    else:
        x = x + 960
        
    if y <= 540:
        y = 540 + y 
    else:
        y = y + 540
        

    print("prediction x, y:" ,x, y, predictions, "\n", "real x, y:", x_r1, y_r1)
    cv2.circle(img, (x,y), 10, (0,0,255), -1)
    cv2.circle(img, (x_real, y_real), 10, (255,0,0), -1)
    cv2.imshow('ss', img)
    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
prediction x, y: 1054 314 [[  94.6431  -226.49776]] 
 real x, y: 126 -156
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
prediction x, y: 1054 400 [[  94.95144 -140.87958]] 
 real x, y: 192 -291
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
prediction x, y: 888 657 [[-72.90849  117.391075]] 
 real x, y: -165 249
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
prediction x, y: 1110 422 [[ 150.99446 -118.64571]] 
 real x, y: 153 -111
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
prediction x, y: 302 635 [[-658.8979    95.99001]] 
 real x, y: -174 -339
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
prediction x, y: 1023 168 [[  63.78663 -372.9479 ]] 
 real x, y: 48 -318
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
prediction x, y: 1015 333 [[  55.377377 -207.27963 ]] 
 real x, y: 45 -264
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
prediction x, y: 1070 596 [[110.64641  56.14554]] 
 real x, y: 183 -78
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
prediction x, y: 678 402 [[-282.90668 -138.5403 ]] 
 real x, y: -477 

In [5]:
model_rap = tf.keras.models.load_model('mouse_attack_fhd_normalized.h5')
model_rap.compile(optimizer='adam',
              loss='mean_squared_error')

model_rap.fit(X_train, y_train, epochs=70, validation_data=(X_val, y_val))
model_rap.save('mouse_attack_fhd_normalized2.h5')

Epoch 1/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - loss: 822823.1875 - val_loss: 92092.2734
Epoch 2/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 68653.7891 - val_loss: 53925.4062
Epoch 3/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 43373.2266 - val_loss: 42781.8867
Epoch 4/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 27863.2031 - val_loss: 32953.3594
Epoch 5/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 15417.3379 - val_loss: 29833.1309
Epoch 6/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 9142.4775 - val_loss: 31822.6719
Epoch 7/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 8245.0713 - val_loss: 34205.4922
Epoch 8/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 6534.7549 - val_loss: 32656.0000
Epoch 9/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 6415.5928 - val_loss: 32802.3438
Epoch 10/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 4967.8770 - val_loss: 33157.5547
Epoch 11/70
21/21 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - loss: 4251.7808 - val_loss: 34